In [943]:
#Read Input & Assign Variables
import time
import csv
import math
with open('speed_to_data_map.csv', newline='') as mapping:
    speed_map = tuple(csv.reader(mapping))
    
speed_data_map = []
for i in range (0,len(speed_map)):
    speed_data_map = (*speed_data_map, int(speed_map[i][1]))


In [944]:
def rank(data,K,init_speed,factor,instance):
    index_map = []
    time = []
    
    for i in range (0,K):
        if instance == 0:
            time.append(data[i]/init_speed[i])
        else:
            time.append(data[i]/(init_speed[i]*(1-(factor[i]/100))))
    
    
    for i in range (0,K):
        if max(time)!=0:
            max_index = (time.index(max(time)))+1
            index_map.append(max_index)
            time[max_index-1] = -1
    return index_map

In [945]:
def target_factor(M,init_speed,factor,total_data,index_map,users,flag,avg_speed,instance,data_remaining):
    sent_data = 0
    collocated_factor_list = [0]*M
    for i in range (0,users):
        collocated_factor_list[i] = factor[index_map[i]-1]/100
    
    for i in range (0,users):
        collocated_factor = collocated_factor_list[i] * sum((collocated_factor_list[:i]+collocated_factor_list[i+1:]))
        speed = init_speed[index_map[i]-1] * (1-collocated_factor)
        if collocated_factor > 1:
            speed = 0
        speed_floor = math.floor(speed + 10e-6)
        sent_data = sent_data + speed_data_map[speed_floor]
        if flag == 1:
            data_remaining[index_map[i]-1] = data_remaining[index_map[i]-1] - speed_data_map[speed_floor]
            if (data_remaining[index_map[i]-1] < 0):
                data_remaining[index_map[i]-1] = 0
            avg_speed[index_map[i]-1] = ((avg_speed[index_map[i]-1])*instance[index_map[i]-1] + speed)/(instance[index_map[i]-1]+1)
            instance[index_map[i]-1] = instance[index_map[i]-1] + 1

    target_curr = (sent_data/total_data)*100
    return target_curr


In [946]:
def write(tc,M,N,grid,avg_speed,best_speed,data_loss,total_data,alpha,start_time):
    with open(str(tc)+'.csv', 'w', newline='') as file:
        writer = csv.writer(file)
    
        for i in range (0,M):
            write_data = []
            for j in range (0,N):
                if (grid[j][i] == 0):
                    write_data.append("-")
                else:
                    write_data.append("U"+str(grid[j][i]))
            writer.writerow(write_data)
            
        penalty = sum(list(data_loss))/total_data
        data_loss.append(penalty)
        writer.writerow(data_loss)
        
        obj_func = sum(list(avg_speed))/best_speed
        avg_speed.append(obj_func)
        writer.writerow(avg_speed)
        
        score = []
        score.append(obj_func - (alpha*penalty))
        writer.writerow(score)
        
        execution_time = []
        temp = math.floor((time.time() - start_time)*1000)
        execution_time.append(temp)
        writer.writerow(execution_time)
        
                
            

In [947]:
def main():
    for tc in range (1,14):
        start_time = time.time()
        with open('tc'+str(tc)+'.csv', newline='') as input:
        #with open('toy_testcase.csv', newline='') as input:
            data = tuple(csv.reader(input))

        M = int(data[0][0])
        N = int(data[0][1])
        K = int(data[1][0])
        alpha = float(data[2][0])
        
        init_speed = ()
        data_size = ()
        factor = ()
        
        for i in range (3,3+K):
            init_speed = (*init_speed, int(data[i][1]))
            data_size = (*data_size, int(data[i][2]))
            factor = (*factor, int(data[i][3]))
     
        total_data = sum(list(data_size))
        best_speed = sum(list(init_speed))
        data_remaining = list(data_size)

        grid = [[0]*M]*N
        avg_speed = [0]*K
        instance = [0]*K
        data_loss = [0]*K
        
        target_ideal = 100/N
        margin_low = 0.99
        margin_high = 2.15

        
        for i in range (0,N):       #Columns 
            user_index_map = [0]*M
            index_map_temp = [0]*M
            for j in range (0,M):
                rank_index_map = rank(data_remaining.copy(),K,init_speed,factor,j)         #shows the index of largest and the subsequent largest data_size 
                #print("rank: ", rank_index_map)
                flag = 0
                for k in range (0,j+1):
                    if sum(list(data_remaining))>0:
                        index_map_temp[k] = rank_index_map[-k]
                #print("temp index", index_map_temp)
                target_factor_temp = target_factor(M,init_speed,factor,total_data,index_map_temp,j+1,flag,avg_speed,instance,data_remaining)
                #print("target_temp: ", target_factor_temp)
                if (margin_low*target_ideal <= target_factor_temp <= margin_high*target_ideal): #or (((sum(list(data_remaining)))/total_data)*100 <= target_ideal):
                    flag = 1
                    user_index_map = index_map_temp.copy()
                    #print("flagged index", user_index_map)
                    target_factor(M,init_speed,factor,total_data,user_index_map,j+1,flag,avg_speed,instance,data_remaining)
                    grid[i] = user_index_map.copy()
                    break
            print("data_rem:", data_remaining)
        data_loss = data_remaining.copy()
        write(tc,M,N,grid,avg_speed,best_speed,data_loss,total_data,alpha,start_time)

In [948]:
if __name__ == '__main__':
    main()
    


index_map: [0, 0, 0, 0]
rank_map: [2, 8, 3, 4, 6, 5, 1, 7]
index_map: [2, 0, 0, 0]
rank_map: [3, 2, 8, 7, 4, 5, 6, 1]
index_map: [3, 0, 0, 0]
rank_map: [3, 2, 8, 7, 4, 5, 6, 1]
data_rem: [0, 8000, 2545, 5400, 4900, 5100, 4200, 8700]
index_map: [0, 0, 0, 0]
rank_map: [2, 8, 4, 6, 5, 7, 3]
index_map: [2, 0, 0, 0]
rank_map: [2, 8, 7, 4, 5, 6, 3]
index_map: [2, 0, 0, 0]
rank_map: [2, 8, 7, 4, 5, 6, 3]
data_rem: [0, 4766, 0, 5400, 4900, 5100, 4200, 8700]
index_map: [0, 0, 0, 0]
rank_map: [8, 2, 4, 6, 5, 7]
data_rem: [0, 4766, 0, 5400, 4900, 5100, 4200, 3583]
index_map: [0, 0, 0, 0]
rank_map: [2, 4, 6, 5, 7, 8]
index_map: [2, 0, 0, 0]
rank_map: [2, 7, 4, 5, 6, 8]
index_map: [2, 0, 0, 0]
rank_map: [2, 7, 4, 5, 6, 8]
data_rem: [0, 994, 0, 5400, 4900, 5100, 4200, 0]
index_map: [0, 0, 0, 0]
rank_map: [4, 6, 5, 7, 2]
data_rem: [0, 994, 0, 14, 4900, 5100, 4200, 0]
index_map: [0, 0, 0, 0]
rank_map: [6, 5, 7, 2, 4]
data_rem: [0, 994, 0, 14, 4900, 0, 4200, 0]
index_map: [0, 0, 0, 0]
rank_map: [5, 7, 

IndexError: list index out of range